# Extracting Experiment Powers
This notebook aids in the extraction of powers from the power log files.

In [1]:
import os 
POWER_DIR='/home/nsultana'

### Scanning through power log
Pre-processing the power log to grab (approximately) the time that is used for the experiments prevents numpy from having to parse days worth of data, and reduces waiting time 

In [2]:
from io import StringIO
import pandas as pd
import numpy as np
import os

def to_datetime(t):
    if isinstance(t, float):
        t = pd.to_datetime(t * 1e9).tz_localize('UTC')
    if isinstance(t, str):
        t = pd.to_datetime(t)
    return t

def open_at_time(fname, start_time, end_time=None, jump_bytes=64000):
    ''' Jumps backwards from the end of the file looking for the first matching timestamps.
    File is readable from this location.
    If end_time is present, reads the contents between start_time and end_time into a file-like buffer.
    '''
    # Must open in binary mode to enable .seek to arbitrary locations
    f = open(fname, 'rb')
    start_time, end_time = to_datetime(start_time), to_datetime(end_time)
    
    end_loc = None
    for i in range(1, 100000):
        f.seek(-jump_bytes * i, os.SEEK_END)
        firstline = f.readline()
        line = f.readline().decode('utf-8')
        strtime = ' '.join(line.split(' ')[:2])
        curtime = pd.to_datetime(strtime)
        if curtime <= start_time:
            print("Found location in file: %s"%line)
            f.seek(-len(line), 1)
            
            if end_loc is not None:
                rd_size = end_loc - f.tell()
                print("Reading into buffer")
                sio = StringIO(f.read(rd_size).decode('utf-8'))
                print("Done.")
                f.close()
                return sio
            else:
                f2 = open(fname, 'r')
                f2.seek(f.tell())
                f.close()
                return f2
        if end_time is not None and curtime > end_time:
            end_loc  = f.tell()
            
    print("Jumped 1000000 times and still didn't find it")
    f.close()
    return open(fname, 'r')

### Read the power log
Power log is reduced to only the time points at which the power changes.
It is cleaned and inserted into a pandas dataframe

In [3]:
import os
def read_power_log(num, start_time, end_time):
    start_time, end_time = to_datetime(start_time), to_datetime(end_time)
    fname = os.path.join(POWER_DIR, 'power_%d.out' % num)
    f = open_at_time(fname, start_time, end_time)
    
    # For some reason this is infinitely faster than using read_csv
    #     df = pd.read_csv(f, sep=' ', error_bad_lines=True, header=None,
    #                      names=['date', 'time', 'device', 'ip', 'power'],
    #                      dtype={'device': str, 'ip': str, 'power': str},
    #                      date_parser=date_parser,
    #                      parse_dates=[[0,1]], infer_datetime_format=True, engine='c',
    #                      low_memory=False)
    
    # Read into a numpy array
    arr = np.loadtxt(f, delimiter=' ', dtype=object)
    df = pd.DataFrame(arr, columns=['date', 'time', 'device', 'ip', 'power'])
    date_time = df.date.astype(object) + ' ' + df.time.astype(object)
    df['date_time'] = pd.to_datetime(date_time.astype(str))
    del df['date']
    del df['time']

    # Discard any rows with missing power values
    df.power = pd.to_numeric(df.power, errors='coerce')
    df = df[~df.power.isna()]
    
    # Error on power monitors sometimes reports very high power values
    df = df[df.power < 10000]
    
    # Filter based on the provided start time
    df = df[df.date_time >= pd.to_datetime(start_time)]
    if end_time is not None:
        df = df[df.date_time <= pd.to_datetime(end_time)]
    print("Read %d power values" % len(df))
    
    # Reduce to only the time-points where the power changes
    df = df[df.power.diff() != 0]
    df = df.reset_index(drop=True)
    
    return df

def read_all_power_logs(n, start_time, end_time=None):
    ''' Reads power logs numbered 0:(n-1) '''
    out = []
    for num in range(n):
        out.append(read_power_log(num, start_time, end_time))
    out = list(filter(lambda x: len(x) > 1, out))
    return out

def read_experiments_powers(events_sets, experiments):
    ''' events_sets[group][experiment][trial][event]'''
    print(experiments)
    mintime = min([ev[experiments[0]][0][0]['time_'] for ev in events_sets])
    maxtime = max([ev[experiments[-1]][-1][-1]['time_'] for ev in events_sets])
    return read_all_power_logs(10, mintime, maxtime)

### Labeling powers during experiment
Using shremote logs, powers are combined into a single dataframe with an added `exp_time` column, which gives the amount of elapsed time since the start of the experiment

In [4]:
EXP_LEN = pd.Timedelta(50, 's')

def experiment_powers(events, dfs, buff = pd.Timedelta(15, 's')):
    starts, ends = [], []
    
    for exp_events in events:
        # NB: Assumes the start of the experiment is the /second/ `tcpreplay` 
        replay_events = filter(lambda x: x['name_'] == 'tcpreplay', exp_events)
        start = list(replay_events)[-1]['time_']
        start = pd.to_datetime(start * 1e9).tz_localize('UTC') - buff
        starts.append(start)
        end = start + EXP_LEN + buff * 2
        ends.append(end)
        
    minstart = min(starts)
    maxend = max(ends)
        
    dfs_exp = []
    for df in dfs:
        df = df[(df.date_time > minstart) & (df.date_time < maxend)]
        for i, (start, end) in enumerate(zip(starts, ends)):
            dfi = df[(df.date_time > start) & (df.date_time < end)]
            dfi = dfi[dfi.power < 1e3]
            dfi = dfi.reset_index(drop=True)
            dfi['exp_time'] = (dfi.date_time - start - buff).apply(lambda x: x.total_seconds())
            dfi.date_time -= start
            dfi["trial"] = i
            dfs_exp.append(dfi)
    return pd.concat(dfs_exp, ignore_index=True)

def all_experiment_powers(events, powers, buff = pd.Timedelta(15, 'seconds')):
    all_dfs = []
    for exp, events in events.items():
        print("Read", exp)
        df = experiment_powers(events, powers)
        df['experiment'] = exp
        all_dfs.append(df)
    return pd.concat(all_dfs, ignore_index=True)

### Loading experiment logs
Need to read the timestamps associated with the experiments to load and label the right data

In [5]:
import json

EXPERIMENTS = ['baseline', 'hc', 'drop', 'fec', 'kv']

def load_experiment_events(exp, n, base_dir):
    ''' Loads experiments 1:n labeled `exp` in directory `base_dir` '''
    events = []
    for i in range(1, n+1):
        exp_file = os.path.join(base_dir, exp, '%s_%d' % (exp, i), 'event_log.json')
        with open(exp_file) as f:
            events.append(json.load(f))
    return events

def load_events(base_dir, n=10, experiments=EXPERIMENTS):
    events = {}
    for exp in experiments:
        exp_events = load_experiment_events(exp, n, base_dir)
        events[exp] = exp_events
    return events

# Preprocess and save data

In [6]:
POWER_SAVE_DIR = '.'
def save_experiment_powers(base_dirs, n=10, experiments=EXPERIMENTS):
    events = {}
    for label, base_dir in base_dirs.items():
        events[label] = load_events(base_dir, n, experiments)
       
    powers = read_experiments_powers(events.values(), experiments)
    
    exps_powers = {}
    for label, exp_events in events.items():
        exp_powers = all_experiment_powers(exp_events, powers)
        exp_powers.to_pickle(os.path.join(POWER_SAVE_DIR, '%s.pickle' % label))
        exps_powers[label] = exp_powers
    return exps_powers

In [7]:
def load_experiment_powers(label):
    return pd.read_pickle(os.path.join(POWER_SAVE_DIR, '%s.pickle' % label))

In [9]:
save_experiment_powers(dict(
    fpga = 'data/e2e_output_500k_fpga_hc_11',
    tofino = 'data/e2e_output_500k_tofino_hc_11'
));

['baseline', 'hc', 'drop', 'fec', 'kv']
Found location in file: 2019-08-21 03:06:10.021598157+00:00 fpga12/1 (192.168.1.115): 27.125

Reading into buffer
Done.
Read 170849 power values
Found location in file: 2019-08-11 23:27:30.834352528+00:00 dcomp1 (192.168.1.108): 106.155

Read 0 power values
Found location in file: 2019-08-21 03:06:47.541570991+00:00 tofino (192.168.1.117): 111.090

Reading into buffer
Done.
Read 170994 power values
Found location in file: 2019-08-21 03:07:32.194541658+00:00 tclust2 (192.168.1.111): 123.170

Reading into buffer
Done.
Read 169648 power values
Found location in file: 2019-08-21 03:07:41.781920569+00:00 tclust4 (192.168.1.114): 104.290

Reading into buffer
Done.
Read 209326 power values
Found location in file: 2019-08-21 03:08:04.999408168+00:00 fpga12/2 (192.168.1.113): 26.225

Reading into buffer
Done.
Read 170918 power values
Found location in file: 2019-08-21 03:07:08.867241037+00:00 fpga13/0 (192.168.1.105): 26.980

Reading into buffer
Done.
Rea

{'fpga':          device                ip    power       date_time   exp_time  trial  \
 0      fpga12/1  (192.168.1.115):   27.110 00:00:01.762563 -13.237437      0   
 1      fpga12/1  (192.168.1.115):   27.105 00:00:02.810665 -12.189334      0   
 2      fpga12/1  (192.168.1.115):   27.115 00:00:03.685390 -11.314609      0   
 3      fpga12/1  (192.168.1.115):   27.130 00:00:04.732500 -10.267500      0   
 4      fpga12/1  (192.168.1.115):   27.125 00:00:05.839482  -9.160517      0   
 ...         ...               ...      ...             ...        ...    ...   
 32226    arista  (192.168.1.118):  109.395 00:01:10.025621  55.025621      9   
 32227    arista  (192.168.1.118):  109.285 00:01:11.076213  56.076213      9   
 32228    arista  (192.168.1.118):  109.160 00:01:12.112506  57.112507      9   
 32229    arista  (192.168.1.118):  109.090 00:01:12.992435  57.992436      9   
 32230    arista  (192.168.1.118):  109.060 00:01:14.034746  59.034747      9   
 
       experiment 

In [10]:
save_experiment_powers(dict(
    cpu = 'data/e2e_output_500k_cpu_hc_11'
), 10, ['baseline', 'hc']);

['baseline', 'hc']
Found location in file: 2019-08-27 17:52:01.548115960+00:00 fpga12/1 (192.168.1.115): 27.160

Reading into buffer
Done.
Read 28264 power values
Found location in file: 2019-08-11 23:27:30.834352528+00:00 dcomp1 (192.168.1.108): 106.155

Read 0 power values
Found location in file: 2019-08-27 17:52:14.190235602+00:00 tofino (192.168.1.117): 110.380

Reading into buffer
Done.
Read 28305 power values
Found location in file: 2019-08-27 17:51:34.869971898+00:00 tclust2 (192.168.1.111): 121.060

Reading into buffer
Done.
Read 27644 power values
Found location in file: 2019-08-27 17:52:53.556819522+00:00 tclust4 (192.168.1.114): 115.265

Reading into buffer
Done.
Read 34348 power values
Found location in file: 2019-08-27 17:51:16.949050155+00:00 fpga12/2 (192.168.1.113): 26.475

Reading into buffer
Done.
Read 28097 power values
Found location in file: 2019-08-27 17:51:22.687451232+00:00 fpga13/0 (192.168.1.105): 26.970

Reading into buffer
Done.
Read 28326 power values
Found

{'cpu':          device                ip    power       date_time   exp_time  trial  \
 0      fpga12/1  (192.168.1.115):   27.130 00:00:00.081767 -14.918232      0   
 1      fpga12/1  (192.168.1.115):   27.170 00:00:01.140010 -13.859990      0   
 2      fpga12/1  (192.168.1.115):   27.150 00:00:02.189545 -12.810454      0   
 3      fpga12/1  (192.168.1.115):   27.165 00:00:03.239181 -11.760819      0   
 4      fpga12/1  (192.168.1.115):   27.150 00:00:04.160768 -10.839231      0   
 ...         ...               ...      ...             ...        ...    ...   
 12901    arista  (192.168.1.118):  108.855 00:01:10.806564  55.806565      9   
 12902    arista  (192.168.1.118):  108.950 00:01:11.857343  56.857343      9   
 12903    arista  (192.168.1.118):  108.770 00:01:12.899695  57.899696      9   
 12904    arista  (192.168.1.118):  108.825 00:01:13.793356  58.793357      9   
 12905    arista  (192.168.1.118):  108.880 00:01:14.841664  59.841665      9   
 
       experiment  